In [ ]:
import os
import tensorflow as tf
import tensorflow_recommenders as tfrs
import tensorflow_model_analysis as tfma

from tfx import components
from tfx.proto import example_gen_pb2, trainer_pb2, pusher_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing


In [ ]:
PIPELINE_ROOT = "pipeline"
SCHEMA_PIPELINE_NAME = "movies-rec"
SERVING_MODEL_DIR = "serving_model_dir"

In [ ]:
MOVIE_DATA_ROOT = "data/movie"
RATING_DATA_ROOT = "data/rating/"

In [ ]:
context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

In [ ]:
movie_output = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=9),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=1),
    ])
)

movie_example_gen = components.CsvExampleGen(
    input_base=MOVIE_DATA_ROOT,
    output_config=movie_output
)
context.run(movie_example_gen)

In [ ]:
rating_output = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=9),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=1),
    ])
)

rating_example_gen = components.CsvExampleGen(
    input_base=RATING_DATA_ROOT,
    output_config=rating_output
)
context.run(rating_example_gen)

In [ ]:
movie_stats_gen = components.StatisticsGen(
    examples=movie_example_gen.outputs["examples"]
)
context.run(movie_stats_gen)

In [ ]:
context.show(movie_stats_gen.outputs["statistics"])

In [ ]:
rating_stats_gen = components.StatisticsGen(
    examples=rating_example_gen.outputs["examples"]
)
context.run(rating_stats_gen)

In [ ]:
context.show(rating_stats_gen.outputs["statistics"])

In [ ]:
movie_schema_gen = components.SchemaGen(
    statistics=movie_stats_gen.outputs["statistics"]
)
context.run(movie_schema_gen)

In [ ]:
rating_schema_gen = components.SchemaGen(
    statistics=rating_stats_gen.outputs["statistics"]
)
context.run(rating_schema_gen)